# 3. Slater determinants

An $N$-particle Slater determinant (with $1 \le N \le L$) is specified via an isometry $\Phi \in \mathbb{C}^{L \times N}$:
\begin{equation*}
\ket{\Phi}_{\text{SD}} = a_{\varphi^{(0)}}^\dagger \cdots a_{\varphi^{(N-1)}}^\dagger \ket{\text{vac}}
\end{equation*}
where the vector $\varphi^{(j)} \in \mathbb{C}^L$ is the $j$-th column of $\Phi$.

In [1]:
import fermi_relations as fr
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import expm

In [2]:
# random number generator
rng = np.random.default_rng(44)

The following example shows how to construct a Slater determinant as a state vector:

In [3]:
# number of modes L
nmodes = 7

# number of particles N
nptcl = 3

# random isometry, defining the "orbital" basis states
orbitals = unitary_group.rvs(nmodes, random_state=rng)[:, :nptcl]

# construct Slater determinant as a state vector
phi = fr.slater_determinant(orbitals)
phi

array([ 0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.02193987+0.0679756j ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        , -0.14506554-0.04280196j,
        0.        +0.j        , -0.01433093-0.11377571j,
       -0.04088634-0.00138997j,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.29324186-0.1026945j ,
        0.        +0.j        ,  0.22527561+0.15303841j,
        0.07191127-0.11609649j,  0.        +0.j        ,
        0.        +0.j        , -0.04339005+0.13667909j,
        0.11880916+0.08643296j,  0.        +0.j        ,
       -0.02184265+0.07005918j,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,

In [4]:
# must be normalized
np.linalg.norm(phi)

np.float64(0.9999999999999998)

In [5]:
# must be an eigenstate of the number operator
np.linalg.norm(fr.total_number_op(nmodes) @ phi - nptcl * phi)

np.float64(0.0)

A consequence of Thouless' theorem (David J. Thouless, Stability conditions and nuclear rotations in the Hartree-Fock theory, [Nucl. Phys. 21, 225-232 (1960)](https://doi.org/10.1016/0029-5582(60)90048-1)) is that the unitary transformation generated by a free-fermion Hamiltonian (see previous notebook) maps Slater determinants to Slater determinants (within the same particle sector).

For a given Hermitian matrix $h \in \mathbb{C}^{L \times L}$, we define the corresponding Hamiltonian acting on the whole Fock space as
\begin{equation*}
H = \sum_{j,k=0}^{L-1} h_{jk} a_j^{\dagger} a_k.
\end{equation*}
The corresponding orbital basis rotation is then given by the unitary matrix $u = e^{-i h}$, and the one on the whole Fock space as $U = e^{-i H}$. Applying the relation (see previous notebook)
\begin{equation*}
U a_{\varphi}^{\dagger} U^{\dagger} = a_{u \varphi}^{\dagger}
\end{equation*}
to the above definition of a Slater determinant (and that $U \ket{\text{vac}} = \ket{\text{vac}}$) leads to
\begin{equation*}
U \ket{\Phi}_{\text{SD}} = U a_{\varphi^{(0)}}^\dagger U^{\dagger} U a_{\varphi^{(1)}}^\dagger U^{\dagger} \cdots U a_{\varphi^{(N-1)}}^\dagger U^{\dagger} U \ket{\text{vac}} = a_{u \varphi^{(0)}}^\dagger a_{u \varphi^{(1)}}^\dagger \cdots a_{u \varphi^{(N-1)}}^\dagger \ket{\text{vac}} = \ket{u \Phi}_{\text{SD}}.
\end{equation*}
Here, $u \Phi$ is understood as a matrix-matrix multiplication.

In [6]:
# random single-particle base change matrix as matrix exponential
h = fr.crandn((nmodes, nmodes), rng)
h = 0.5*(h + h.conj().T)
u = expm(-1j*h)

# corresponding base change matrix on the whole Fock space
clist, alist, _ = fr.construct_fermionic_operators(nmodes)
hfull = sum(h[i, j] * (clist[i] @ alist[j])
            for i in range(nmodes)
            for j in range(nmodes))
ufull = expm(-1j * hfull.toarray())

# numerically verify the above equation
np.linalg.norm(ufull @ phi - fr.slater_determinant(u @ orbitals))

np.float64(7.90354560393188e-16)